<a target="_blank" href="https://colab.research.google.com/github/kircherlab/ISMB-2025_IGVF-MPRA-Tutorial/blob/main/05_sequence_models/02_ism_and_tfmodisco.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Interpreting Models with In-Silico Mutagenesis

With tools like TF-MoDISco for motif discovery, we will investigate important transcription factor binding motifs using models trained on sequence data that predict activity. We will assess whether these motifs exert activating or repressing effects by comparing the activity of sequences with and without these identified motifs in the cell type of interest.

## Notebook Preparation

### Load Libraries and Data

In [ ]:
import os
import random
import gzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # Suppress TensorFlow warnings

In [ ]:
!pip install modisco-lite

Download the data if not already present.

In [ ]:
%%bash
mkdir -p 02_data

if [ ! -f "02_data/ultra_joint.fa.gz" ]; then
    wget -O 02_data/ultra_joint.fa.gz https://github.com/kircherlab/ISMB-2025_IGVF-MPRA-Tutorial/raw/refs/heads/main/05_sequence_models/02_data/ultra_joint.fa.gz
else
    echo "File exists"
fi

### Prepare Sequences from FASTA for DNN Model

Set seeds to make code reproducible. This is always a good idea when working with TensorFlow. The seed of 42 is arbitraty.

In [ ]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

Load sequences for which we want to generate contribution maps. Print one example entry. The sequences are in FASTA format. Each entry contains two lines: the first is the sequence ID, the second is the sequence. Note that we have removed the ">" from the output printed below.

In [ ]:
seq_file = "02_data/ultra_joint.fa.gz"

with gzip.open(seq_file, 'rt') as f:
    file_content = f.read()

sequence_entries = file_content.split('>')

print(sequence_entries[3])

Write IDs and sequences into Python lists.

In [ ]:
sequence_ids = []
sequences = []

for i in range(1, len(sequence_entries)):
    entry = sequence_entries[i]
    if "\n" not in entry:
        print(entry)
        raise ValueError("Entry missing newline between ID and sequence.")
    seq = entry.split("\n")[1]
    if "Y" in seq or "R" in seq:
        continue
    sequence_ids.append(entry.split("\n")[0])
    sequences.append(seq)

# Show sequence number 10 as an example
print("ID list entry example:", sequence_ids[9])
print("Sequence list entry example:", sequences[9])

To pass the sequences to our model, we need to "one-hot encode" them. We'll load the necessary function.

In [ ]:
def one_hot_encode(seq):
    nucleotide_dict = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'N': [0, 0, 0, 0]
    }
    return np.array([nucleotide_dict[nuc] for nuc in seq])

Apply one-hot encoding to our sequences. The output demonstrates how one-hot encoding looks.

In [ ]:
sequences_tok = [one_hot_encode(seq) for seq in sequences]

print("Sequence list entry example (first 5 bases):\n", sequences[9][:5])
print("Sequence list entry example one-hot encoded (first 5 bases):\n", sequences_tok[9][:5])

Convert the lists into a pandas DataFrame. This helps with removing duplicates and inspecting the data.

In [ ]:
df_IDs_seq = pd.DataFrame({
    "ID": sequence_ids,
    "seq": sequences_tok,
    "seq_org": sequences
})

df_IDs_seq = df_IDs_seq.drop_duplicates(subset="ID")

print("The ID column of the dataframe:\n", df_IDs_seq["ID"])
print("\nThe original sequence column of the dataframe:\n", df_IDs_seq["seq_org"])
print("\nThe one-hot encoded sequence column of the dataframe:\n", df_IDs_seq["seq"])

Convert the one-hot encoded sequences to TensorFlow tensors for input to our pre-trained model.

In [ ]:
input_seq_all = tf.convert_to_tensor(df_IDs_seq["seq"].to_list())

### Load Model

Load our pretrained model. Here, we use the MPRAnn architecture trained on sequences and experimental MPRA activity from the MPRAultra study (K562 dataset). Ensure your data matches the model's expected sequence length.

In [ ]:
%%bash
mkdir -p 02_data

if [ ! -f "02_data/h5" ]; then
    wget -O 02_data/MPRAnn.tar.gz https://github.com/kircherlab/ISMB-2025_IGVF-MPRA-Tutorial/raw/refs/heads/main/05_sequence_models/02_data/MPRAnn.h5
else
    echo "File exists"
fi

In [ ]:
model = keras.models.load_model("02_data/MPRAnn.h5")

## Model Interpretation Using Attribution Maps and TF-MoDISco

### Sequences

We will initialize numpy arrays to store the attribution maps of the sequences based on our pre-trained model, as well as the one-hot encoded sequences. Both are required inputs for TF-MoDISco. For demonstration, we only process a subset of sequences to keep computation feasible.

## Sequences for TF-MoDISco

In [ ]:
numberOfSeqs = np.asarray(input_seq_all).shape[0] // 6000  # Use a subset for speed
seq_length = np.asarray(input_seq_all).shape[1]

seqs_tfmodisco_format = np.zeros([numberOfSeqs, seq_length, 4], dtype=bool)
for i in range(numberOfSeqs):
    seqs_tfmodisco_format[i] = input_seq_all[i].numpy().astype(bool)

np.savez('02_data/Sequences.npz', seqs_tfmodisco_format)

Load the one-hot encoded sequences and inspect them. The array shape matches the contribution score array. Data type is bool.

In [ ]:
print(seqs_tfmodisco_format)

In [ ]:
seq_file = "02_data/Sequences.npz"
seqs = np.load(seq_file)
seqs = seqs.f.arr_0.astype(bool)

print(seqs[9])
print(seqs.shape)
print(seqs.dtype)

### Attribution Scores

Now we compute the attribution (contribution) scores using in-silico mutagenesis (ISM). For each sequence, we generate all possible single-nucleotide variants (SNVs) and predict their activity using our model.

In [ ]:
hypothetical_contribution_scores = np.zeros([numberOfSeqs, seq_length, 4])

for i in range(numberOfSeqs):
    print(f"{i+1} of {numberOfSeqs}")
    listOfMutSeqs = []
    for j in range(seq_length):
        temp_seq = input_seq_all[i].numpy().copy()
        temp_seq[j, :] = 0
        for k in range(4):
            temp_seq[j, k] = 1
            listOfMutSeqs.append(tf.convert_to_tensor(temp_seq.copy()))
            temp_seq[j, k] = 0
    tensorOfMutSeqs = tf.convert_to_tensor(listOfMutSeqs)
    predictions_alt = model.predict(tensorOfMutSeqs, batch_size=256, verbose=0)
    for j in range(seq_length):
        for k in range(4):
            hypothetical_contribution_scores[i, j, k] = predictions_alt[j*4+k, 0]

np.savez("02_data/hypothetical_contribution_scores.npz", hypothetical_contribution_scores)

## Attribution Maps

This section introduces attribution maps. First, we load the file with the contribution scores generated above.

### Attribution Maps in General

Attribution maps highlight bases or motifs that are important for the model's prediction of a given sequence. Let's look at the contribution scores as calculated above:

In [ ]:
contrib_file = "02_data/hypothetical_contribution_scores.npz"
scores = np.load(contrib_file)
scores = scores.f.arr_0.astype(np.float32)

print(scores[9])

A widely used approach is to subtract the prediction of the reference sequence from all predictions (see [Kelley et al.](https://pubmed.ncbi.nlm.nih.gov/27197224/)).

In [ ]:
for i in range(scores.shape[0]):
    ref_seq = scores[i] * seqs[i]
    ref_pred = np.max(ref_seq[np.nonzero(ref_seq)])
    scores[i] = scores[i] - ref_pred
print(scores[9])

Let's look at the attribution map of a single sequence. For each position, there are 4 entries (A, C, G, T). The value indicates the importance of the base at that position for the measured MPRA activity, based on the model. The array shape is (number of sequences, sequence length, 4).

In [ ]:
print(scores[9])
print(scores.shape)
print(scores.dtype)

We now use the logomaker library to plot a selected attribution map (here, the first one). Importantness is expressed by scaling the bases according to the maximal possible prediction loss.

In [ ]:
scores_for_map = np.abs(scores - np.min(scores, axis=2, keepdims=True))
scores_for_map *= seqs

df_hypo_contri = pd.DataFrame(scores_for_map[9], columns=['A', 'C', 'G', 'T'])
print(df_hypo_contri)

In [ ]:
%matplotlib inline

In [ ]:
import logomaker

fig = plt.figure(figsize=(8, 1), dpi=300)
ax1 = fig.add_subplot(111)
PWM_logo = logomaker.Logo(df_hypo_contri, shade_below=.5, fade_below=.5, ax=ax1)
plt.savefig("02_data/example_contrib_map.png", dpi=300)
plt.show()

### Attribution Maps for TF-MoDISco

For TF-MoDISco, it is recommended to mean-normalize the contribution scores (see [TF-MoDISco example](https://github.com/kundajelab/tfmodisco/blob/master/examples/simulated_TAL_GATA_deeplearning/Generate%20Importance%20Scores.ipynb)).

In [ ]:
scores_for_modisco = scores - np.mean(scores, axis=2, keepdims=True)
np.savez("02_data/scores_for_modisco.npz", hypothetical_contribution_scores)

Let's plot the mean-normalized hypothetical scores used for TF-MoDISco:

In [ ]:
df_hypo_contri = pd.DataFrame(scores_for_modisco[9], columns=['A', 'C', 'G', 'T'])
fig = plt.figure(figsize=(8, 1), dpi=300)
ax1 = fig.add_subplot(111)
PWM_logo = logomaker.Logo(df_hypo_contri, shade_below=.5, fade_below=.5, ax=ax1)
plt.savefig("02_data/example_contrib_map.png", dpi=300)
plt.show()

And the actual contribution scores resulting from it:

In [ ]:
df_hypo_contri = pd.DataFrame(scores_for_modisco[9] * seqs[9], columns=['A', 'C', 'G', 'T'])
fig = plt.figure(figsize=(8, 1), dpi=300)
ax1 = fig.add_subplot(111)
PWM_logo = logomaker.Logo(df_hypo_contri, shade_below=.5, fade_below=.5, ax=ax1)
plt.savefig("02_data/example_contrib_map.png", dpi=300)
plt.show()

Note that both ways of plotting actual contribution scores result from the same data and include very similar information.

# Using TF-MoDISco

## Setting Up the Environment

Now we can set up the environment.

In [ ]:
import modiscolite

## Running TF-MoDISco

Above, we learned how to interpret a model's representation of a given sequence. However, with tens of thousands of sequences, you don't want to look at each one individually. TF-MoDISco summarizes the model's representation across all sequences.

Now that we have prepared all the arrays for TF-MoDISco, we can run it. The result depends on the parameters you use. Here, we use only a subset of the data for speed.

In [ ]:
pos_patterns, neg_patterns = modiscolite.tfmodisco.TFMoDISco(
    sliding_window_size=8,
    flank_size=8,
    min_metacluster_size=20,
    target_seqlet_fdr=0.1,
    hypothetical_contribs=scores_for_modisco,
    one_hot=seqs,
    max_seqlets_per_metacluster=20000,
    trim_to_window_size=10,
    n_leiden_runs=2,
    initial_flank_to_add=3,
    final_min_cluster_size=30,
    verbose=True
)

modiscolite.io.save_hdf5(str(contrib_file[0]) + ".h5", pos_patterns, neg_patterns, window_size=8)

print(pos_patterns)
print(neg_patterns)

Finally, we can run the command-line version of TF-MoDISco to generate an HTML report. The example below uses the full dataset. The JASPAR database is used for motif annotation.

In [ ]:
!modisco report --help


In [ ]:
%%bash
modisco report \
-i 02_data/modisco_hypothetical_contribution_scores_all.npz.h5 \
-o 02_data/tfmodisco_report/ \
-s 02_data/tfmodisco_report/ \
-m 02_data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme_nice.txt

After creating a copy of the file `motifs.html` in the directory above the output folder, we can explore the TF-MoDISco results.

In [ ]:
# Demo example



In [ ]:
!cp 02_data/tfmodisco_report/motifs.html motifs.html
from IPython.display import HTML
HTML(filename='motifs.html')

Finally we show you the TF-Modisco report on the complete sample (not only 10).

In [ ]:
!tar -xzf 02_data/tfmodisco_report_all.tar.gz -C 02_data/
!sed -i 's/tfmodisco_report_MPRAultra_all\//02_data\/tfmodisco_report_all\//g' 02_data/tfmodisco_report_all/motifs.html 
!cp 02_data/tfmodisco_report_all/motifs.html motifs_all.html
HTML(filename='motifs_all.html')